In [1]:
import os
import pandas as pd
import numpy as np
import re

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', -1)

In [6]:
file_intermediate = "/home/laura/ANALYSIS/VARIANT_CALLING/Deletion_UK_remove/Compare/2021-01-21_Deletion_UK_remove/Deletion_UK_remove.revised_intermediate.tsv"
bed_file = "/home/laura/DATABASES/Anotacion/COVID/UK_deletion.bed"

In [54]:
def bed_to_df(bed_file):
    """
    Import bed file separated by tabs into a pandas df
    -Handle header line
    -Handle with and without description (If there is no description adds true or false to annotated df)
    """
    header_lines = 0
    #Handle likely header by checking colums 2 and 3 as numbers
    with open(bed_file, 'r') as f:
        next_line = f.readline().strip()
        line_split = next_line.split(None) #This split by any blank character
        start = line_split[1]
        end = line_split[2]
        while not start.isdigit() and not end.isdigit():
            header_lines = header_lines + 1
            next_line = f.readline().strip()
            line_split = next_line.split(None) #This split by any blank character
            start = line_split[1]
            end = line_split[2]

    if header_lines == 0:
        df = pd.read_csv(bed_file, sep="\t", header=None) #delim_whitespace=True
    else:
        df = pd.read_csv(bed_file, sep="\t", skiprows=header_lines, header=None) #delim_whitespace=True

    df = df.iloc[:,0:4]
    df.columns = ["#CHROM", "start", "end", "description"]
        
    return df

In [55]:
df = pd.read_csv(file_intermediate, sep="\t")

In [56]:
df.shape

(40, 6)

In [60]:
def remove_bed_positions(df, bed_file):
    bed_df = bed_to_df(bed_file)
    for _, row in df.iterrows():
        position_number = int(row.Position.split("|")[2])
        if any(start <= position_number <= end for (start, end) in zip(bed_df.start.values.tolist(), bed_df.end.values.tolist())):
            df = df[df.Position != row.Position]
    return df


In [63]:
df = remove_bed_positions(df, bed_file)

In [64]:
df.shape

(35, 6)

In [65]:
df

,Position,N,Samples,20517531,21013270,20513699
0,NC_045512.2|C|241|T,3,"20517531,21013270,20513699",1.00,1.0,1.0
1,NC_045512.2|C|913|T,3,"20517531,21013270,20513699",1.00,0.93,1.0
2,NC_045512.2|C|3037|T,3,"20517531,21013270,20513699",1.00,1.0,1.0
3,NC_045512.2|C|3267|T,3,"20517531,21013270,20513699",1.00,0.96,1.0
4,NC_045512.2|C|5388|A,3,"20517531,21013270,20513699",1.00,1.0,1.0
5,NC_045512.2|C|5986|T,3,"20517531,21013270,20513699",1.00,0.95,1.0
6,NC_045512.2|T|6954|C,3,"20517531,21013270,20513699",1.00,1.0,0.99
7,NC_045512.2|C|6968|T,3,"20517531,21013270,20513699",1.00,1.0,0.99
11,NC_045512.2|C|14408|T,3,"20517531,21013270,20513699",1.00,1.0,1.0
12,NC_045512.2|C|14676|T,3,"20517531,21013270,20513699",1.00,0.93,1.0
